In [ ]:
!pip install transformers==4.28.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Load models

#### Load aragpt model

In [ ]:
!unzip -q "/content/drive/MyDrive/aragpt_final_one.zip" -d "/content/"

In [ ]:
aragpt_folder_path = '/content/aragpt_model'
!mkdir -p "$aragpt_folder_path"

In [ ]:
aragpt_tokenizer = GPT2Tokenizer.from_pretrained(
    'aubmindlab/aragpt2-base'
    )
aragpt_model = GPT2LMHeadModel.from_pretrained(
    aragpt_folder_path,
    pad_token_id = aragpt_tokenizer.eos_token_id
    )

In [ ]:
aragpt_generator_ = pipeline(
    'text-generation',
    model = aragpt_folder_path,
    tokenizer='aubmindlab/aragpt2-base'
    )

### Load the gpt model

In [ ]:
!unzip -q "/content/drive/MyDrive/gpt_for_poems (2).zip" -d "/content/"

In [ ]:
folder_path = '/content/gpt_model'
!mkdir -p "$folder_path"

In [ ]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained(
    folder_path,
    pad_token_id = gpt_tokenizer.eos_token_id
    )

In [ ]:
english_generator = pipeline(
    'text-generation',
    model = folder_path,
    tokenizer = gpt_tokenizer
    )

### Load the "Uneversal Sentence Encoder (USE)" model for embedding.

In [ ]:
#Load the pre-trained model#
embed = hub.load(
    "https://tfhub.dev/google/universal-sentence-encoder-large/5"
    )

# Define some usefull functions

In [ ]:
def calculate_similarity(embed1, embed2):
    # Define two vectors
    vector1 = np.array(embed1)
    vector2 = np.array(embed2)

    # Reshape the vectors to have a 2D shape
    vector1 = vector1.reshape(1, -1)
    vector2 = vector2.reshape(1, -1)

    # Calculate the cosine similarity
    similarity = cosine_similarity(vector1, vector2)

    return similarity

In [ ]:
def generate_embedding_for_prediction(seed_line, generator_):

    #Generate text based on a line
    predicted_line = generator_(seed_line, max_length = 20)[0]['generated_text']
    predicted_embedding = embed( [predicted_line] )

    return [predicted_embedding]

In [ ]:
def generate_embedding_for_arabic_prediction(seed_line: str):
    return generate_embedding_for_prediction(seed_line, aragpt_generator_)

def generate_embedding_for_english_prediction(seed_line: str):
    return generate_embedding_for_prediction(seed_line, english_generator)

# Evaluate the English one.

### Get embedding for the verses in the test dataset because each one except the first one will be the ground truth prediction for the verses before it.

### Read the test data

In [ ]:
# Read the text file
with open('test_dataset_en.txt', 'r') as file:
    lines = file.readlines()

# Create a DataFrame with each row as a sample
df = pd.DataFrame({'text': lines})


df['text'] = df['text'].astype(str)

# Remove empty rows
df = df[df['text'] != '']

df = df.head(150)

### Get embeddings for the ground truth

In [ ]:
english_embeddings = embed(
    df['text'].to_list()
)

In [ ]:
df['embeddings'] = [embedded_sentence for embedded_sentence in english_embeddings]

In [ ]:
df

### Get embeddings for the predicted text

In [ ]:
df['predicted_embedding'] = df['text'].map(generate_embedding_for_english_prediction)

In [ ]:
df

### Find the cosine similiraty between embeddings.


In [ ]:
df['similarity'] = [i-i for i in range(df.shape[0])]
df = df[df['predicted_embedding'].notna()]
df.shape[0]


In [ ]:
for i in range(1, df.shape[0] - 1):
    predicted_embedding = df['predicted_embedding'].iloc[i][0]
    ground_truth_embedding = df['predicted_embedding'].iloc[i + 1][0]

    df['similarity'].iloc[i] = calculate_similarity(predicted_embedding, ground_truth_embedding)


#### get some stats about similarities between the ground truth with the predicted embeddings.

In [ ]:
df['similarity'].describe()

In [ ]:
print(
    english_generator(
        'dark storms his genial powers controul\n',
        max_length = 80
        )[0]['generated_text']
      )

## Model Enhancement:
  1. fine-tune the model on more data.
  2. Increase number of epochs during the tunning process.

# Evaluate the Arabic one.

In [ ]:
print(
    aragpt_generator_(
        'لا  قلت شعرا ولا سمعت غنا ولا',
        max_length = 80)[0]['generated_text']
      )

### Read the test data

In [ ]:
# Read the text file
with open('test_dataset.txt', 'r') as file:
    lines = file.readlines()

# Create a DataFrame with each row as a sample
df = pd.DataFrame({'text': lines})


df['text'] = df['text'].astype(str)

# Remove empty rows
df = df[df['text'] != '']


In [ ]:
df = df.head(150)
df

### Generate embeddings for the ground truth

In [ ]:
embeddings = embed(
    df['text'].to_list()
)

In [ ]:
df['embeddings'] = [embedded_sentence for embedded_sentence in embeddings]
df['predicted_embedding'] = [i-i for i in range(df.shape[0])]

In [ ]:
for i in range(150):
    df['predicted_embedding'].iloc[i + 1] = generate_embedding_for_arabic_prediction(df['text'].iloc[i])

In [ ]:
df

### Finding similarity between the textual embedding.

In [ ]:
df['similarity'] = [i-i for i in range(df.shape[0])]
df = df[df['predicted_embedding'].notna()]

In [ ]:
for i in range(1, 149):
    predicted_embedding = df['predicted_embedding'].iloc[i][0]
    ground_truth_embedding = df['predicted_embedding'].iloc[i + 1][0]

    df['similarity'].iloc[i] = calculate_similarity(predicted_embedding, ground_truth_embedding)

In [ ]:
df

In [ ]:
df['similarity'].describe()

**Comparing this result with results I had from the english transformer, Aragpt-2 performed better than it.**